In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scipy.stats import pearsonr
from collections import Counter

<IPython.core.display.Javascript object>

In [3]:
def convert_to_float(value):
    try:
        # If it's a string enclosed in single quotes, remove quotes and replace comma with dot
        if isinstance(value, str):
            return float(value.replace(",", "."))
        else:
            return float(value)
    except ValueError as e:
        print(e)
        return float("nan")

<IPython.core.display.Javascript object>

In [4]:
def preprocess_numeric_cols(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)  # Ensure string type for string operations
        df[col] = df[col].str.replace("#", "")  # Remove '#' characters
        df[col] = df[col].str.replace(",", ".")  # Replace ',' with '.' for decimals
    return df

<IPython.core.display.Javascript object>

# Data preparation - 209 plant Z data

### Reading the files and extracting relevant information:

In [5]:
plant = "Z"
xls_files = {}

xls = pd.ExcelFile(
    f"../../../../data/raw/209/EMBRAPII hubIC IACC_CIMENTO_{plant}.xlsx",
    engine="openpyxl",
)
xls_files[plant] = xls

<IPython.core.display.Javascript object>

In [6]:
print("Plant: ", plant, xls.sheet_names)

Plant:  Z ['INSTRUÇÕES', 'Clínquer', '30', '44', '99', '116', '138']


<IPython.core.display.Javascript object>

In [7]:
cement_types_per_plant = []
CEMENT_TYPE_COLS = ["Tipo de cimento", "Classe de resistência"]
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[1, 2])

        if not df[CEMENT_TYPE_COLS].iloc[2:].isna().all().all():
            cement_type = (
                df[CEMENT_TYPE_COLS]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .str.replace("nan", "")
                .unique()
            )

        else:
            cement_type = (
                df[[("Obs.", "Unnamed: 8_level_1")]]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .unique()
            )

        cement_types_per_plant.append((plant, sheet_name, cement_type))
        dataframes.append(df)

<IPython.core.display.Javascript object>

In [8]:
cement_types_per_plant

[('Z', '30', array(['CPIIE40'], dtype=object)),
 ('Z', '44', array(['CPIIF40'], dtype=object)),
 ('Z', '99', array(['CPIII40'], dtype=object)),
 ('Z', '116', array(['CPIIF32'], dtype=object)),
 ('Z', '138', array(['CPVARI'], dtype=object))]

<IPython.core.display.Javascript object>

In [9]:
cements = []
for tup in cement_types_per_plant:
    cements.append(tup[2][0])
Counter(cements)

Counter({'CPIIE40': 1, 'CPIIF40': 1, 'CPIII40': 1, 'CPIIF32': 1, 'CPVARI': 1})

<IPython.core.display.Javascript object>

In [10]:
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[0, 1, 2])
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"] = plant
        dataframes.append(df.drop([0, 1], axis=0).reset_index(drop=True).copy())

<IPython.core.display.Javascript object>

In [11]:
df = pd.concat(dataframes, axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

## Initial Preprocessing

In [12]:
df = pd.concat(
    [
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"],
        df["Dados iniciais"],
        df["Cimento"]["Composição"].rename(
            {"Sulfato de cálcio": "Sulfato de cálcio (composição)"}, axis=1
        ),
        df["Cimento"]["Análise mineralógica"],
        df["Cimento"]["Análise química"],
        df["Característias físicas do cimento"],
        df["Propriedades do cimento"],
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

In [13]:
# df.info()

<IPython.core.display.Javascript object>

### 1. Dropping columns where either there is no data nor it contributes to the analysis
### 2. Renaming columns to a short identifiable name

In [14]:
COLUMNS_TO_DROP = [
    ("Data", "Medida"),
    ("Horário", "Produção"),
    ("Horário", "Medida"),
    ("Obs.", "Unnamed: 148_level_2"),
    ("Tipo de cimento", "Unnamed: 6_level_2"),
    ("Classe de resistência", "Unnamed: 7_level_2"),
    ("Obs.", "Unnamed: 8_level_2"),
    "Obs.",
    ("Obs.", "Unnamed: 148_level_2"),
    ("Unnamed: 141_level_1", "Obs."),
    # ("Unnamed: 139_level_1", "#400"),
]

COLUMNS_TO_RENAME = {
    ("Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"): "Plant",
    ("Cidade", "Unnamed: 1_level_2"): "City",
    ("Data", "Produção"): "Date",  # Arrumar!
    ("Unnamed: 136_level_1", "Blaine"): "Blaine",
    ("Unnamed: 137_level_1", "#200"): "#200",
    ("Unnamed: 138_level_1", "#325"): "#325",
    ("Unnamed: 139_level_1", "#400"): "#400",
    ("Unnamed: 140_level_1", "Massa específica"): "Specific Gravity",
    ("Tempo de pega", "Inicio"): "Initial setting time",
    ("Tempo de pega", "Fim"): "Final setting time",
    ("Resistência à compressão", "1 d"): "CS1",
    ("Resistência à compressão", "3 d"): "CS3",
    ("Resistência à compressão", "7 d"): "CS7",
    ("Resistência à compressão", "28 d"): "CS28",
    "Perda ao fogo": "Loss on Ignition",
    "Resíduo insolúvel": "Insoluble Residue",
    "Clínquer": "Clinker",
    "Fíler calcário": "Calcareous filler",
    "Cinza volante": "Fly ash",
    "Escória": "Slag",
    "Argila calcinada": "Calcined clay",
    "Sulfato de cálcio": "Calcium sulfate",
    "C3S total": "Total C3S",
    "C2S total": "Total C2S",
    "C2S alpha": "Alpha C2S",
    "C2S beta": "Beta C2S",
    "C2S gama": "Gamma C2S",
    "C4AF": "C4AF",
    "C3A": "C3A",
    "C3A cubic": "Cubic C3A",
    "C3A orto": "Orthorhombic C3A",
    "CaO livre": "Free CaO",
    "Portlandita": "Portlandite",
    "Periclasio": "Periclase",
    "Arcanita": "Arcanite",
    "Aphthalita": "Aphthitalite",
    "Langbeinita": "Langbeinite",
    "Bassanita": "Bassanite",
    "Anidrita": "Anhydrite",
    "Calcita": "Calcite",
    "Dolomita": "Dolomite",
    "Quartzo": "Quartz",
    "Muscovita": "Muscovite",
    "Sulfato de cálcio (composição)": "Calcium sulfate (composition)",
}

<IPython.core.display.Javascript object>

### Defining a single variable to identify Cement Type

In [15]:
# Single variable for the Cement Type
df["Cement_Type"] = (
    df[
        [
            ("Tipo de cimento", "Unnamed: 6_level_2"),
            ("Classe de resistência", "Unnamed: 7_level_2"),
            ("Obs.", "Unnamed: 8_level_2"),
        ]
    ]
    .fillna("")
    .astype(str)
    .sum(axis=1)
)

# # Preprocessing
df["Cement_Type"] = df["Cement_Type"].replace("Fibro", "Fibrocimento")

# Preprocessing
df["Cement_Type"] = (
    df["Cement_Type"]
    .str.replace("Expedido", "")
    .str.replace("Linha 9", "")
    .str.replace("Itajaí", "")
    .str.replace("D-G", "")
    .str.replace("D-E", "")
    .str.replace("DF2", "")
    .str.replace("CPIIF40", "CP II-F-40")
    .str.replace("CPIIF40", "CP II-F40")
    .str.replace("CP VARI", "CP V-ARI")
    .str.replace("CPVARI", "CP V-ARI")
    .str.replace("CP V-RSARI", "CP V-ARI RS")
    .str.replace("CP III-RS40", "CP III-40 RS")
    .str.replace("CP III-RS32", "CP III-32 RS")
    .str.replace("CP IV-RS32", "CP IV-32 RS")
    .str.replace("CP III40", "CP III-40")
    .str.replace("CP III32", "CP III-32")
    .str.replace("CP I-S40", "CP I-S-40")
    .str.replace("CP I40", "CP I-40")
    .str.replace("CP II-E32", "CP II-E-32")
    .str.replace("CP II-E40", "CP II-E-40")
    .str.replace("CP II-F FIBRO40", "CP II-F-40")
    .str.replace("CP II-F32", "CP II-F-32")
    .str.replace("CP II-F40", "CP II-F-40")
    .str.replace("CP II-Z32", "CP II-Z-32")
    .str.replace("CP II-Z40", "CP II-Z-40")
    .str.replace("CP IV32", "CP IV-32")
    .str.replace("CP IND", "CP I")
    .str.replace("CPINDCC", "CP I")
    .str.strip()
)

df = df.drop(COLUMNS_TO_DROP, axis=1).rename(COLUMNS_TO_RENAME, axis=1).copy()
df = df.drop([0]).reset_index(drop=True)
df = df.infer_objects()

<IPython.core.display.Javascript object>

### Dropping and rename the columns defined above

### Cleaning and converting numeric columns to float

In [16]:
NUMERIC_COLUMNS = [
    "Calcium sulfate (composition)",
    "Clinker",
    "Calcareous filler",
    "Fly ash",
    "Slag",
    "Calcined clay",
    "Calcium sulfate",
    "Total C3S",
    "Total C2S",
    "Alpha C2S",
    "Beta C2S",
    "Gamma C2S",
    "C4AF",
    "C3A",
    "Cubic C3A",
    "Orthorhombic C3A",
    "Free CaO",
    "Portlandite",
    "Periclase",
    "Arcanite",
    "Aphthitalite",
    "Langbeinite",
    "Bassanite",
    "Anhydrite",
    "Calcite",
    "Dolimita",
    "Quartz",
    "Muscovite",
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "TiO2",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
    "Blaine",
    "#200",
    "#325",
    "#400",
    "Specific Gravity",
    "Final setting time",
    "Initial setting time",
    "CS1",
    "CS3",
    "CS7",
    "CS28",
]

df = preprocess_numeric_cols(df, NUMERIC_COLUMNS)

# Preprocessing to fix numeric columns
df[NUMERIC_COLUMNS] = df[NUMERIC_COLUMNS].map(convert_to_float)

# Converting Date to pandas datetime
df["Date"] = pd.to_datetime(df["Date"])

# Drop rows completely missing
index_to_keep = (
    df.drop(["Plant", "Date", "Cement_Type"], axis=1).dropna(axis=0, how="all").index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

df = df.infer_objects()

could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string 

<IPython.core.display.Javascript object>

## Dropping missing features

### Drop rows completely missing

In [17]:
index_to_keep = (
    df.drop(["Plant", "City", "Date", "Cement_Type"], axis=1)
    .dropna(axis=0, how="all")
    .index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Removing features
 that will be fitted on this data.

1. Removing features with features having ~50% or more of missing values
2. Removing features with zero variance
3. Removing features with more than 70% of zeros

In [18]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Langbeinite,100.000000
Specific Gravity,100.000000
Clinker,100.000000
Calcareous filler,100.000000
Fly ash,100.000000
Slag,100.000000
Calcined clay,100.000000
Calcium sulfate,100.000000
Muscovite,100.000000
Beta C2S,92.867332


<IPython.core.display.Javascript object>

In [19]:
for var, val in (
    (df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).items()
):
    s = f"""\t<tr>
            <td>{var}</td>
            <td>{round(val, 2)}</td>
    \t</tr>"""
    if val >= 40:
        # print(s)
        print(f'"{var}",')

"Langbeinite",
"Specific Gravity",
"Clinker",
"Calcareous filler",
"Fly ash",
"Slag",
"Calcined clay",
"Calcium sulfate",
"Muscovite",
"Beta C2S",
"Gamma C2S",
"Alpha C2S",
"CS1",
"C3A",
"Aphthitalite",
"Total C2S",
"Total C3S",
"Anhydrite",
"Calcium sulfate (composition)",


<IPython.core.display.Javascript object>

<h3>Percentage of missing values:</h3>
<table>
    <th>Plant Z - Full Dataset:</th>
    <th></th>
	<tr>
            <td>Langbeinite</td>
            <td>100.0</td>
    	</tr>
	<tr>
            <td>Specific Gravity</td>
            <td>100.0</td>
    	</tr>
	<tr>
            <td>Clinker</td>
            <td>100.0</td>
    	</tr>
	<tr>
            <td>Calcareous filler</td>
            <td>100.0</td>
    	</tr>
	<tr>
            <td>Fly ash</td>
            <td>100.0</td>
    	</tr>
	<tr>
            <td>Slag</td>
            <td>100.0</td>
    	</tr>
	<tr>
            <td>Calcined clay</td>
            <td>100.0</td>
    	</tr>
	<tr>
            <td>Calcium sulfate</td>
            <td>100.0</td>
    	</tr>
	<tr>
            <td>Muscovite</td>
            <td>100.0</td>
    	</tr>
	<tr>
            <td>Beta C2S</td>
            <td>92.87</td>
    	</tr>
	<tr>
            <td>Gamma C2S</td>
            <td>92.87</td>
    	</tr>
	<tr>
            <td>Alpha C2S</td>
            <td>92.87</td>
    	</tr>
	<tr>
            <td>CS1</td>
            <td>89.87</td>
    	</tr>
	<tr>
            <td>C3A</td>
            <td>61.27</td>
    	</tr>
	<tr>
            <td>Aphthitalite</td>
            <td>57.7</td>
    	</tr>
	<tr>
            <td>Total C2S</td>
            <td>49.86</td>
    	</tr>
	<tr>
            <td>Total C3S</td>
            <td>49.86</td>
    	</tr>
	<tr>
            <td>Anhydrite</td>
            <td>42.72</td>
    	</tr>
	<tr>
            <td>Calcium sulfate (composition)</td>
            <td>42.72</td>
    	</tr>
</table>

In [20]:
FEATRUES_TO_DROP = [
    "Langbeinite",
    "Specific Gravity",
    "Clinker",
    "Calcareous filler",
    "Fly ash",
    "Slag",
    "Calcined clay",
    "Calcium sulfate",
    "Muscovite",
    "Beta C2S",
    "Gamma C2S",
    "Alpha C2S",
    "CS1",
    "C3A",
    "Aphthitalite",
    "Total C2S",
    "Total C3S",
    "Anhydrite",
    "Calcium sulfate (composition)",
]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Removing features with more than 23% of zeros

In [21]:
zero_values = {}
for col in df.select_dtypes(include="number").columns:
    zero_percentages = (df[df[col].eq(0)].shape[0] / df.shape[0]) * 100
    zero_values[col] = zero_percentages

zero_percentages = pd.Series(zero_values, name=f"Zero (%) for {plant}")
zero_percentages = zero_percentages.sort_values(ascending=False)
zero_percentages = zero_percentages.to_frame(name=f"Zero (%) for {plant}")
zero_percentages.style.background_gradient(cmap="Reds")

,Zero (%) for Z
Dolimita,58.915835
#200,51.069900
Free CaO,11.269615
Arcanite,3.994294
#325,1.069900
Bassanite,0.784593
Quartz,0.570613
#400,0.000000
Loss on Ignition,0.000000
Insoluble Residue,0.000000


<IPython.core.display.Javascript object>

<h3>Percentage of zero values:</h3>
<table>
    <th>Plant A - Full Dataset:</th>
    <th></th>
    <tr>
        <td>Dolimita</td>
        <td>58.91</td>
    </tr>
    <tr>
        <td>#200</td>
        <td>51.07</td>
    </tr>
    <tr>
        <td>Free CaO</td>
        <td>11.27</td>
    </tr>    
</table>

In [22]:
FEATRUES_TO_DROP = ["Dolimita", "Free CaO", "#200"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping variable with City and Plant identification

In [23]:
FEATRUES_TO_DROP = ["City", "Plant"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is null

In [24]:
df = df[df["CS28"].notna()]

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is zero

In [25]:
df = df[~df["CS28"].eq(0)]

<IPython.core.display.Javascript object>

In [26]:
df.shape

(1330, 30)

<IPython.core.display.Javascript object>

### Dropping rows where any other variable has a zero value

In [27]:
for col in df.columns:
    df = df[~df[col].eq(0)]

<IPython.core.display.Javascript object>

### Dropping duplicated rows

In [28]:
df.shape

(1242, 30)

<IPython.core.display.Javascript object>

In [29]:
df = df.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [30]:
df = df.loc[df.select_dtypes("number").drop_duplicates().index].reset_index(drop=True)

chemical_vars = [
    "C4AF",
    "Cubic C3A",
    "Orthorhombic C3A",
    "Portlandite",
    "Periclase",
    "Arcanite",
    "Bassanite",
    "Calcite",
    "Quartz",
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
]

df = df.loc[
    df.select_dtypes("number").drop_duplicates(subset=chemical_vars).index
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [31]:
df.shape

(1239, 30)

<IPython.core.display.Javascript object>

### Dropping rows with negative values

In [32]:
df[df.drop(["Date"], axis=1).duplicated()].drop("Date", axis=1).select_dtypes(
    include="number"
).apply(lambda x: x < 0).sum().sort_values(ascending=False)

C4AF                    0.0
Cubic C3A               0.0
CS7                     0.0
CS3                     0.0
Final setting time      0.0
Initial setting time    0.0
#400                    0.0
#325                    0.0
Blaine                  0.0
Insoluble Residue       0.0
Loss on Ignition        0.0
Fe2O3                   0.0
TiO2                    0.0
K2O                     0.0
SO3                     0.0
SiO2                    0.0
Al2O3                   0.0
Na2O                    0.0
MgO                     0.0
CaO                     0.0
Quartz                  0.0
Calcite                 0.0
Bassanite               0.0
Arcanite                0.0
Periclase               0.0
Portlandite             0.0
Orthorhombic C3A        0.0
CS28                    0.0
dtype: float64

<IPython.core.display.Javascript object>

### Removing outliers

In [33]:
df.describe(percentiles=[0.99]).T

,count,mean,min,50%,99%,max,std
Date,1239,2022-04-28 11:51:16.997578496,2021-01-05 00:00:00,2022-05-12 00:00:00,2023-07-13 00:00:00,2023-07-25 00:00:00,NaN
C4AF,1058.0,6.677854,2.86,6.76,9.2286,10.48,1.372882
Cubic C3A,1058.0,4.108715,0.81,4.22,6.4429,7.13,1.136103
Orthorhombic C3A,1058.0,0.334518,0.01,0.28,1.1586,3.02,0.286698
Portlandite,1058.0,0.933894,0.06,0.89,2.15,4.68,0.391272
Periclase,1058.0,3.83603,1.22,3.95,5.5,6.31,0.894477
Arcanite,1058.0,0.711569,0.01,0.67,1.7815,2.18,0.420458
Bassanite,1058.0,1.204868,0.01,1.16,2.8686,3.5,0.642883
Calcite,1058.0,6.090378,2.18,5.675,11.8817,14.44,2.060741
Quartz,1058.0,0.353422,0.01,0.33,0.9843,1.5,0.204403


<IPython.core.display.Javascript object>

In [34]:
df = df.reset_index(drop=True)
df = df[~df["Loss on Ignition"].eq(df["Loss on Ignition"].max())].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [35]:
df.describe(percentiles=[0.99]).T

,count,mean,min,50%,99%,max,std
Date,1238,2022-04-28 04:07:45.266558976,2021-01-05 00:00:00,2022-05-11 12:00:00,2023-07-13 00:00:00,2023-07-25 00:00:00,NaN
C4AF,1057.0,6.67685,2.86,6.76,9.2288,10.48,1.373143
Cubic C3A,1057.0,4.108174,0.81,4.22,6.4432,7.13,1.136505
Orthorhombic C3A,1057.0,0.334494,0.01,0.28,1.1588,3.02,0.286832
Portlandite,1057.0,0.933841,0.06,0.89,2.15,4.68,0.391453
Periclase,1057.0,3.835374,1.22,3.95,5.5,6.31,0.894645
Arcanite,1057.0,0.710956,0.01,0.67,1.782,2.18,0.420183
Bassanite,1057.0,1.204825,0.01,1.16,2.8688,3.5,0.643186
Calcite,1057.0,6.091996,2.18,5.68,11.8836,14.44,2.061044
Quartz,1057.0,0.353396,0.01,0.33,0.9844,1.5,0.204498


<IPython.core.display.Javascript object>

In [36]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
TiO2,35.621971
Arcanite,14.620355
C4AF,14.620355
Calcite,14.620355
Bassanite,14.620355
Quartz,14.620355
Periclase,14.620355
Portlandite,14.620355
Orthorhombic C3A,14.620355
Cubic C3A,14.620355


<IPython.core.display.Javascript object>

## Dropping rows with increased missing values after preprocessing

<h3>Percentage of missing values:</h3>
<table>
    <th>Plant W - Full Dataset:</th>
    <th></th>
    <tr>
        <td>TiO2</td>
        <td>35.83</td>
    </tr>
</table>

In [37]:
FEATRUES_TO_DROP = ["TiO2"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

In [38]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
C4AF,14.620355
Cubic C3A,14.620355
Orthorhombic C3A,14.620355
Portlandite,14.620355
Periclase,14.620355
Arcanite,14.620355
Bassanite,14.620355
Calcite,14.620355
Quartz,14.620355
#325,10.581583


<IPython.core.display.Javascript object>

In [39]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,1238,2022-04-28 04:07:45.266558976,2021-01-05 00:00:00,2021-08-25 06:00:00,2022-05-11 12:00:00,2023-01-11 00:00:00,2023-07-25 00:00:00,NaN
C4AF,1057.0,6.67685,2.86,6.02,6.76,7.53,10.48,1.373143
Cubic C3A,1057.0,4.108174,0.81,3.59,4.22,4.84,7.13,1.136505
Orthorhombic C3A,1057.0,0.334494,0.01,0.11,0.28,0.5,3.02,0.286832
Portlandite,1057.0,0.933841,0.06,0.68,0.89,1.13,4.68,0.391453
Periclase,1057.0,3.835374,1.22,3.49,3.95,4.39,6.31,0.894645
Arcanite,1057.0,0.710956,0.01,0.37,0.67,0.99,2.18,0.420183
Bassanite,1057.0,1.204825,0.01,0.74,1.16,1.61,3.5,0.643186
Calcite,1057.0,6.091996,2.18,4.66,5.68,7.37,14.44,2.061044
Quartz,1057.0,0.353396,0.01,0.2,0.33,0.45,1.5,0.204498


<IPython.core.display.Javascript object>

In [40]:
df.describe(exclude="number").T

,count,unique,top,freq,mean,min,25%,50%,75%,max
Date,1238,NaN,NaN,NaN,2022-04-28 04:07:45.266558976,2021-01-05 00:00:00,2021-08-25 06:00:00,2022-05-11 12:00:00,2023-01-11 00:00:00,2023-07-25 00:00:00
Cement_Type,1238,5,CP II-E-40,666,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238 entries, 0 to 1237
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  1238 non-null   datetime64[ns]
 1   C4AF                  1057 non-null   float64       
 2   Cubic C3A             1057 non-null   float64       
 3   Orthorhombic C3A      1057 non-null   float64       
 4   Portlandite           1057 non-null   float64       
 5   Periclase             1057 non-null   float64       
 6   Arcanite              1057 non-null   float64       
 7   Bassanite             1057 non-null   float64       
 8   Calcite               1057 non-null   float64       
 9   Quartz                1057 non-null   float64       
 10  CaO                   1206 non-null   float64       
 11  MgO                   1206 non-null   float64       
 12  Na2O                  1109 non-null   float64       
 13  Al2O3             

<IPython.core.display.Javascript object>

### Sort the dataset by date

In [42]:
# We first make sure Date column was infered correctly
df["Date"].dtype

dtype('<M8[ns]')

<IPython.core.display.Javascript object>

In [43]:
df = df.sort_values(by="Date")

<IPython.core.display.Javascript object>

## Saving the dataset

<h3>Saving for feature engineering</h3>

In [44]:
df.to_csv("../../../../data/interim/209/z.csv", index=False)

<IPython.core.display.Javascript object>

In [45]:
df.dropna().shape

(899, 29)

<IPython.core.display.Javascript object>